<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

<center> Julia Lane, Clayton Hunter, Brian Kim, Benjamin Feder, Ekaterina Levitskaya, Tian Lou, Lisa Osorio-Copete. 
</center>

# Outcome measurement and imputation

## Introduction

What should you do when you encounter missing values in your data? Unfortunately, there is usually no *right* answer. However, you can try to impute these missing values, providing your best guess for each missing point's true value. Here, you will learn how to implement common imputation methods you can use in approaching missing values in your own work.

### Learning Objectives

* Gain understanding of the concept of measurement error in the context of a cohort's earnings

* Explore options for imputing missing values

* Visualize estimate changes following imputation

In this notebook, you will focus on the earnings of individuals who were the primary recipients of TANF spells ending in 2016Q4 during their first year after this TANF spell completion, particularly in their first and fourth quarters after leaving the TANF program. Recall that in the [Data Exploration](03_Dataset_Exploration.ipynb/#Post-TANF-Employment-Outcomes) notebook, you initially examined the earnings distribution for all members of this cohort who had positive earnings in this time period in Indiana. To evaluate the earnings outcomes of all primary recipients of TANF spells ending in 2016Q4 in Indiana, you need to decide what to do when you cannot find their earnings in the Indiana Unemployment Insurance (UI) wage records. A person may not appear in Indiana's UI wage records for several reasons:
- The person is unemployed. 
- The person is out of labor force, e.g., schooling, childcare, etc...
- The person was employed outside of the state.
- The person's job is not covered in UI wage records, e.g.,self-employed, independent contractors, federal government works, etc. <a href='https://www.nap.edu/read/10206/chapter/11#294'>(Hotz and Scholz, 2002)</a>

You will explore the resulting earnings outcomes after applying different earnings imputation methods. The methods covered in this notebook include:
- Dropping all "missing" values
- Filling in zero for people who do not have records in Indiana UI wage records data 
- Substituting missing values with the average earnings of people who received TANF benefits from the same county and have the same gender
- Regression imputation

## R Setup and Database Connection

Before you begin, you need to run the code cells below to import the libraries and connect to our PostgreSQL database.

In [ ]:
#database interaction imports
library(DBI)
library(RPostgreSQL)

# for data manipulation/visualization
library(tidyverse)

# scaling data, calculating percentages, overriding default graphing
library(scales)

In [ ]:
# create an RPostgreSQL driver
drv <- dbDriver("PostgreSQL")

# connect to the database
con <- dbConnect(drv,dbname = "postgresql://stuffed.adrf.info/appliedda")

## Brief Manipulation: Isolating Earnings during first quarter after exit

Before we start performing imputation, we need to do some quick data manipulation to isolate earnings from the first quarter after each individual's exit. Luckily, the first quarter after exit is simply where `quarter` is equal to one.

In [ ]:
# read table into R
qry <- "
select *
from ada_tdc_2020.cohort_2016_earnings
"
df_2016_wages <- dbGetQuery(con, qry)

In [ ]:
# add in quarter after graduation
q1_wages <- df_2016_wages %>%
    filter(quarter == 1)

# see unique values of quarter
q1_wages %>%
    distinct(quarter)

Because we will want to estimate the total wages for each `ssn` in this quarter, not necessarily their wages per employer, let's aggregate `q1_wages` to find the total earnings for each member of this cohort in the entire quarter.

In [ ]:
# aggregate on ssn
q1_wages <- q1_wages %>%
    group_by(ssn) %>%
    summarize(tot_wages = sum(wages)) %>%
    ungroup()

In [ ]:
nrow(q1_wages)

In [ ]:
q1_num <- q1_wages %>%
    summarize(n=n_distinct(ssn))

cat('The total number of primary recipients with positive earnings during their first quarter after ending their 2016Q4 TANF spell:', q1_num$n)

To see the percentage of our cohort represented in `q1_wages`, let's load in our original cohort into R.

In [ ]:
# 2016Q4 cohort with most recent case information
qry <- "
SELECT *
FROM ada_tdc_2020.cohort_2016
"

#read into R as df
df_2016 <- dbGetQuery(con,qry)

In [ ]:
cat('That is', percent(q1_num$n/nrow(df_2016), .01), 'of the study cohort.')

<h3 style="color:red">Checkpoint 1: Identifying Earnings in the Fourth Quarter after ending a TANF spell in 2016Q4 </h3>

Given the code above, create a data subset `q4_wages` that contains all earnings for the cohort in their fourth quarter after leaving the TANF program. How many members of the cohort had positive earnings in this quarter? Did you expect this number to be higher or lower than the number in the first quarter?

## Add leavers without positive earnings for Q1

Our current data frame, `q1_wages`, only contains individuals with positive earnings in their first quarter after exit in Indiana. Let's add in members of our cohort who did not appear in Indiana's wage records during this time period, as well the additional variables from the original cohort table to better describe the individuals. This will let us easily analyze different earnings distributions in the cohort's first quarter after exit as we progress throughout this notebook.

We can do so by using a `left_join()` of the original cohort, `df_2016`, to `q1_wages`, as this will add in one row for each `ssn` in the original cohort that was not included in `q1_wages`.

In [ ]:
# add in employment outcomes for all of those in the original cohort
q1_all_wages <- df_2016 %>%
    left_join(q1_wages, c("ssn"))

# see data frame
head(q1_all_wages)

As a quick check, we can see if the number of individuals in `q1_all_wages` that either have or do not have null wages makes sense given the total number of individuals in the cohort that were in `q1_wages`. We can do so by adding in an indicator variable if the `wages` column was null for each potential wage record in `q1_all_wages`, and then counting the number of distinct individuals based on this new variable.

In [ ]:
# employment outcomes for all of those in our original cohort
q1_all_wages %>%
    mutate(wage_ind = ifelse(is.na(tot_wages), 'no', 'yes')) %>%
    group_by(wage_ind) %>%
    summarize(n=n_distinct(ssn))

In [ ]:
# check number of individuals in q1_wages
q1_num$n

We can see that these numbers make sense. If they did not add up, chances are there was an issue with the details of the join.

For future usage, let's add the gender to each individual in `q1_all_wages`. This variable can be accessed within the table `person_month` in the `in_fssa` schema. Let's load the contents of this table into R in preparation for the join, but only the contents for those in the original cohort.

In [ ]:
# load person_month into R
qry <- "
select *
from in_fssa.person_month
where ssn in (select ssn from ada_tdc_2020.cohort_2016)
"
person_month <- dbGetQuery(con, qry)

In [ ]:
# check to see every individual is in person_month
n_distinct(person_month$ssn)

However, there may be more than one row in `person_month` corresponding to each individual, as `person_month` contains a record for every month the individual was associated with any TANF case.

In [ ]:
# see how many rows we have in person_month
nrow(person_month)

We can now `left_join()` `person_month` to `q1_all_wages` to find the corresponding gender value for each individual in `q1_all_wages`. To ensure we are just selecting these specific columns pertaining to the proper `caseid`, `month` and `tanf_end` indicator from `master_person`, we will include them as well as variables on which to join in our code below.

> Additionally, we will de-select all repetitive columns in `q1_all_wages` and `person_month` that were not included in the join conditions, as they will not provide any statistical power for the following analysis.

In [ ]:
# join and de-select unneccessary variables and see the names of the columns
q1_all_wages %>%
    left_join(person_month, c('ssn', 'caseid', 'month', 'tanf_end')) %>%
    select(-c(ends_with('.x'), ends_with('.y'))) %>%
    head()

In [ ]:
# update q1_all_wages
q1_all_wages <- q1_all_wages %>%
    left_join(person_month, c('ssn', 'caseid', 'month', 'tanf_end')) %>%
    select(-c(ends_with('.x'), ends_with('.y')))

Just to confirm, we can check to see if the number of rows in `q1_all_wages` is equal to the number of rows in `df_2016`, the original cohort, as each individual in the original cohort should correspond to a single row regardless of employment status.

In [ ]:
nrow(df_2016) == nrow(q1_all_wages)

Let's also check to see if we have any missing values for our `gender`, `dob_yr`, and `county` variables. If so, let's fill these in as `unknown` so these rows will not be dropped in future analyses.

In [ ]:
# see number of na values for gender and dob_yr
colSums(is.na(q1_all_wages[c('gender', 'dob_yr', 'county')]))

<h3 style="color:red">Checkpoint 2: Replicate for Q4</h3>

Create a data frame `q4_all_wages` that mirrors `q1_all_wages` except for Q4. Feel free to add in as many code cells as you deem necessary.

## Impute Wage Values

Now that we have confirmed that our `q1_all_wages` dataframe is ready to use for testing our imputation methods, we can get started. To recall, here are the four methods we will be trying out in this notebook:
- Dropping all "missing" values
- Filling in zero for people who do not have records in Indiana UI data
- Filling in missing values with the average earnings of people who received benefits from the same county and have the same gender
- Regression

### 1. Drop All Missing Values

First, let's look at the earnings outcomes during first quarter after TANF exits in 2016Q4 when we drop all missing earnings values. Here, by ignoring potentially non-missing values, we are hoping that they mirror the same distribution as the present one. Although this is fairly common, you should **never, ever, ever** use this method in practice. 

In [ ]:
# drop missing values
q1_no_missing <- q1_all_wages %>%
    filter(!is.na(tot_wages))

In [ ]:
# see earnings distribution
summary(q1_no_missing$tot_wages)

<h4 style="color:red">Checkpoint 3: Replicate for Q4</h4>

What does the earnings distribution look like for Q4 when you drop missing values?

### 2. Fill in Missing Values with Zero

Next, let's see how the earnings distribution shifts when we encode all missing earnings outcomes as 0. Here, we are assuming that all missing earnings are due to unemployment.

In [ ]:
# fill all null tot_wages with 0
q1_wages_zero <- q1_all_wages %>%
    mutate(tot_wages = ifelse(is.na(tot_wages), 0, tot_wages)) 

In [ ]:
# Take a look at the distribution. How does it vary from the distribution you get in method 1?
summary(q1_wages_zero$tot_wages)

In [ ]:
cat('Average earnings if missing wages are dropped is $', round(mean(q1_no_missing$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed as 0 is $', round(mean(q1_wages_zero$tot_wages), 2), sep = '', '.')

<h4 style="color:red">Checkpoint 4: Replicate for Q4</h4>

What does the earnings distribution look like for Q4 when you fill missing values with zero?

### 3. Fill in Missing Values with County/Gender Mean Earnings

Now, instead of either ignoring missing values or assuming the earnings are 0, we will try imputing missing earnings for each individual as the average quarterly earnings of the other individuals in our cohort of the same `gender` and `county`.

Here, our strategy is as follows:
- Find mean earnings for each `county` by `gender`
- Merge the mean earnings for each `county` by `gender` to quarterly earnings for each member of the cohort by `county`/`gender` combination
- If the earnings are null, replace with mean earnings of `county`/`gender` combination

In [ ]:
names(q1_all_wages)

In [ ]:
# mean earnings by gender/county grouping
q1_all_wages %>%
    group_by(gender, county) %>%
    summarize(mean_wages = mean(tot_wages, na.rm=T)) %>%
    head()

In [ ]:
#mean earnings by gender/county grouping saved
q1_county_gend <- q1_all_wages %>%
    group_by(gender, county) %>%
    summarize(mean_wages = mean(tot_wages, na.rm=T)) %>%
    ungroup()

Now, we will merge the two DataFrames, `q1_county_gend` and `q1_all_wages` using `inner_join`.
> Note: `left_join()` would also work in this case.

In [ ]:
# see if join works
q1_all_wages %>%
    inner_join(q1_county_gend, by=c('gender', 'county')) %>%
    head()

In [ ]:
# save join results to q1_joined_county_gend
q1_joined_county_gend <- q1_all_wages %>%
    inner_join(q1_county_gend, by=c('gender', 'county'))

Now, we can add a new column to `q1_joined_county_gend` to include the mean `county`/`gender` wage if the individual did not appear in the Indiana UI wage records data. 

In [ ]:
# see if mutation works as designed
q1_joined_county_gend %>%
    mutate(imputed_wages = ifelse(is.na(tot_wages), mean_wages, tot_wages)) %>%
    select(tot_wages, mean_wages, imputed_wages) %>%
    head()

In [ ]:
# save mutation to q1_major_gend_impute
q1_county_gend_impute <- q1_joined_county_gend %>%
    mutate(imputed_wages = ifelse(is.na(tot_wages), mean_wages, tot_wages))

In using this method, there is a chance we cannot impute missing values for all individuals in the cohort. If `imputed_wages` is still `NA`, we can assume there were no individuals in the cohort with non-missing earnings with the same `county`/`gender` combination.

In [ ]:
# see if any still don't have imputed earnings
q1_county_gend_impute %>%
    filter(is.na(imputed_wages)) %>%
    summarize(n=n())

Unfortunately, it seems as though we do not have available earnings for every combination of `gender` and `county`. For the sake of the exercise, we will ignore the earnings of those whose we could not impute using this method.

In [ ]:
summary(q1_county_gend_impute$imputed_wages)

<h4 style="color:red">Checkpoint 5: Replicate for Q4</h4>
Impute missing earnings values as the mean earnings of individuals in the cohort with the same `gender` and `county` fields. What does the earning distribution look like? For how many individuals could you not impute values using this method?

### 4. Regression imputation

We can also use regression to try to get more accurate earnings values. We will build a regression equation from the obervations for which we know the earnings, then use the equation to predict the missing earnings values. This is, in effect, an extension of the mean imputation by subgroup. Here, we will use demographic information of graduates such as birth year, gender, race, and education level.

> Note: We will not be checking the assumptions associated with linear regressions, as this example is aimed at merely displaying how to use a linear regression for imputation. If you plan on using regression imputation, please check all assumptions before employing a predictive model.

In [ ]:
# subset to variables included in regression analysis
q1_reg <- q1_all_wages %>%
    select(ssn, tot_wages, dob_yr, gender, hispan, native, asian, black, hawaia, white, edul)

In [ ]:
# see types of the variables
glimpse(q1_reg)

As per the data dictionary, all education levels from `01`-`11` correspond to the same grouping. Let's reflect that in `q1_reg`.

In [ ]:
# morph edul variable to reflect data dictionary
q1_reg <- q1_reg %>%
    mutate(edul = ifelse(as.numeric(edul) < 12, '01-11', edul))

# see result
q1_reg %>%
    select(edul) %>%
    head()

Since we will build the model using the members of our cohort with non-missing wages, we will split `q1_reg` into two datasets, one for testing (`q1_wages_na`) and one for training (`q1_wages_pred`).

In [ ]:
# split into training and testing sets
# don't need tot_wages because they are null 
q1_wages_na <- q1_reg %>%
    filter(is.na(tot_wages)) %>%
    select(-c(tot_wages))

q1_wages_pred <- q1_reg %>%
    filter(!is.na(tot_wages))

The model creation process for a linear regression can be done using the `lm()` function. The variable we are trying to predict is on the left-hand side of `lm()` before the `~`, and the predictors are all of the variables on the right-hand side of the `~`.

In [ ]:
# run model and fit coefficients
q1_wages_model <- lm(tot_wages ~ dob_yr + gender + hispan + native + asian + black + hawaia + white + edul, data = q1_wages_pred)

Now that we have fit coefficients for each of the predictors in the model, we can predict the `tot_wages` variable for the test set using `predict()`.

In [ ]:
# predict earnings for test set
pred_earnings <- data.frame(tot_wages = predict(q1_wages_model, newdata=q1_wages_na))

In [ ]:
# see predicted earnings
head(pred_earnings)

Because the output for `predict()` retains the same order of rows from `q1_wages_na`, we can add the `tot_wages` variable from `pred_earnings` into the existing `q1_wages_na` data frame.

In [ ]:
# see updated data frame with predicted earnings
cbind(q1_wages_na, pred_earnings) %>% 
    head()

In [ ]:
# save updated data frame
q1_wages_na_w_earnings <- cbind(q1_wages_na, pred_earnings)

Finally, before we can see the effects of the imputation method, we need to combine our training set, which already has `tot_wages`, with our testing set and its predicted `tot_wages`. 

In [ ]:
# combine training and testing sets
rbind(q1_wages_na_w_earnings, q1_wages_pred) %>% 
    head()

In [ ]:
# save combined training and testing sets
q1_reg_earnings <- rbind(q1_wages_na_w_earnings, q1_wages_pred)

Now we can see the entire earnings distribution for the cohort after applying regression imputation.

In [ ]:
# see earnings distribution for full cohort
summary(q1_reg_earnings$tot_wages)

In [ ]:
# see earnings distribution for imputed portion of cohort
summary(q1_wages_na_w_earnings$tot_wages)

<font color=red><h4> Checkpoint 6: Add in `citzn` and re-run the regression</h4></font> 

When you add `citzn` into the regression, how does the earnings distribution compare to the one using the previous linear regression to impute values?

## Visualizing Earnings Distributions

We can quickly determine if these different imputation methods significantly altered the pre-imputation wage distribution by visualizing the overall earnings distribution. Plotting side-by-side boxplots can be an effective choice. To do so, we need to bind the earnings from all of these methods by rows, meaning they must have the same columns. For the sake of simplicity, we will have three columns in this data frame:

- `ssn`, the person identifier
- `tot_wages`, cumulative earnings in first quarter post-exit
- `method`, type of imputation method

In [ ]:
# adapt q1_no_missing
q1_no_missing %>%
    select(ssn, tot_wages) %>% head()

q1_no_missing <- q1_no_missing %>%
    select(ssn, tot_wages) %>%
    mutate(method = 'remove missing')

In [ ]:
# adapt q1_reg_earnings
q1_reg_earnings%>%
    select(ssn, tot_wages) %>% head()

q1_reg_earnings <- q1_reg_earnings %>%
    select(ssn, tot_wages) %>%
    mutate(method = 'regression')

In [ ]:
# adapt q1_wages_zero
q1_wages_zero %>%
    select(ssn, tot_wages) %>% head()

q1_wages_zero <- q1_wages_zero %>%
    select(ssn, tot_wages) %>%
    mutate(method = 'zero')

In [ ]:
#adapt q1_county_gend_impute
q1_county_gend_impute %>% 
    select(ssn, imputed_wages) %>% 
    rename(tot_wages = imputed_wages) %>% 
    head()

q1_county_gend_impute <- q1_county_gend_impute %>%
    select(ssn, tot_wages) %>%
    mutate(method = 'mean')

Now that these methods all have the same column names, we can feed them into `rbind()`.

In [ ]:
# combine earnings from all methods
all_methods <- rbind(q1_county_gend_impute, q1_reg_earnings, q1_no_missing, q1_wages_zero)

Instead of plotting the earnings distributions of each method one at a time, we can plot them all in a side-by-side fashion by using the `facet_grid()` function as we did in the Data Visualization [notebook](05_Data_Visualization.ipynb/#Distribution-of-quarterly-wages-by-degree-rank).

In [ ]:
# boxplot of all methods
all_methods %>%
    ggplot(aes(x=tot_wages, y ='')) +
    geom_boxplot() + 
    facet_grid(method ~ .) +
    labs(
        title = "REDACTED",
        x='Quarter 1 Earnings',
        y='Method',
        caption = 'Source: Indiana TANF and UI Wage Records data'
    ) +
    theme_minimal()

## Multiple histograms

We can also look at the differences in the earnings distribution by looking at side-by-side histograms. Instead of using the `geom_` layer `geom_boxplot()`, we will use `geom_histogram()`.

In [ ]:
all_methods %>%
    ggplot(aes(x=tot_wages)) +
    geom_histogram() + 
    facet_grid(method ~ .) +
    labs(
        title = 'REDACTED',
        y = 'Density',
        x='Quarterly Wages',
        caption = 'Source: Indiana TANF and UI Wage Records data'
    ) +
    theme_minimal()

### (Optional) Advanced: Using machine learning to impute values

To impute values, we can also use machine learning algorithms such as `K-nearest Neighbors` and `Decision Trees`. The principle behind `K-nearest Neighbors` is quite simple: the missing values can be imputed by values of "closest neighbors" - as approximated by other, known, features. 

For example, if we had cases where the data on earnings of some graduates was completely missing, we could approximate their earnings by referring to other characteristics which could be shared by major group (their 'closest neighbors' in terms of characteristics).

The algorithm calculates the distance between the input values (the missing values) and helps to identify the nearest possible value based on other features (such as known characteristics of the closest major group). Imputing missing data using machine learning has become a research hotbed, and there are plenty of papers covering the various algorithms if you are curious.